In [1]:
#載入套件
import pandas as pd
import numpy as np

In [2]:
#匯入資料，並將資料分成訓練組及測試組
train_data = pd.read_csv('train.csv')
train_data['Gender'] = np.where(train_data['Gender']=='Male', 1, 2)
train_data['Geography'] = np.where(train_data['Geography']=='Spain', 1, np.where(train_data['Geography']=='France', 2, 3))
test_data = pd.read_csv('test.csv')
test_data['Gender'] = np.where(test_data['Gender']=='Male', 1, 2)
test_data['Geography'] = np.where(test_data['Geography']=='Spain', 1, np.where(test_data['Geography']=='France', 2, 3))

In [3]:
#因為Exited是之後要預測的變數，所以X軸不用
X_train = train_data.drop(['Exited', 'Surname', 'RowNumber', 'CustomerId'],axis=1)
y_train = train_data['Exited']
X_test = test_data.drop(['Surname', 'RowNumber', 'CustomerId'],axis=1)

In [4]:
X_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,720,1,1,38,5,114051.97,2,0,1,107577.29
1,682,2,2,54,4,62397.41,1,1,0,113088.60
2,672,2,2,31,5,119903.67,1,1,1,132925.17
3,592,1,2,40,4,104257.86,1,1,0,110857.33
4,753,1,1,42,5,120387.73,1,0,1,126378.57
...,...,...,...,...,...,...,...,...,...,...
7995,568,2,2,35,6,121079.60,2,1,1,124890.50
7996,602,3,2,45,7,145846.07,1,1,0,99276.02
7997,679,1,2,43,5,132810.01,1,1,0,130780.85
7998,715,2,1,38,4,118729.45,1,0,0,95484.52


In [4]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

#建立分類器
clf = RandomForestClassifier(n_estimators=20)

#給定參數搜尋範圍：list or distribution
param_dist = {"max_depth": [3, None],                     #给定list
              "max_features": sp_randint(1, 11),          #给定distribution
              "min_samples_split": sp_randint(2, 11),     #给定distribution
              "bootstrap": [True, False],                 #给定list
              "criterion": ["gini", "entropy"]}           #给定list

#用RandomSearch+CV選取超參數
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5)
clf=random_search.fit(X_train, y_train)
clf.best_params_ 

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 2,
 'min_samples_split': 8}

In [5]:
#使用隨機森林
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=20,
                            bootstrap= True,
                            criterion= 'entropy',
                            max_depth= None,
                            max_features= 2,
                            min_samples_split= 10)

#從訓練組資料中建立隨機森林模型
model.fit(X_train,y_train)

#預測測試組結果是否Exited
y_pred = model.predict(X_test)

In [6]:
#輸出結果
df = test_data.loc[:, 'RowNumber']
df = pd.DataFrame(df)
df.insert(1, column="Exited", value=y_pred)
df.to_csv('output_RandomSearchCV.csv')